In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import
import pandas as pd
import numpy as np
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE, Isomap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, Normalizer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, mean_absolute_error
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.multioutput import MultiOutputRegressor
import random
import copy

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/BigData_proj/model2022/model
file_root = './fifa_dataset_without_value.csv'
df_old =  pd.read_csv(file_root)

/content/drive/MyDrive/Colab Notebooks/BigData_proj/model2022/model


In [ ]:
df = pd.read_csv(file_root)
#date 형식 변환
#df['date'] = [i.replace('-','') for i in df['date']]
#all_list : tournament,home_team, away_team
all_list = ((df['tournament'].unique()).tolist() + (df['home_team'].unique()).tolist() + (df['away_team'].unique()).tolist())

uni_len = '***'.join(all_list)
nat_enc = tf.keras.preprocessing.text.Tokenizer(num_words=len(all_list),
    filters='',
    lower=False,
    split = '***')
nat_enc.fit_on_texts([uni_len])
nat_dic = nat_enc.word_index

In [ ]:
#값을 enumerate로 변환
for step, val in enumerate(df['tournament']):
    df['tournament'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['home_team']):
    df['home_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['away_team']):
    df['away_team'].iloc[step] = (nat_dic.get(val))
for step, val in enumerate(df['neutral']):
    if df['neutral'].iloc[step] == False:
        df['neutral'].iloc[step] = 0
    if df['neutral'].iloc[step] == True:
        df['neutral'].iloc[step] = 1

In [ ]:
df.head()

,Unnamed: 0,home_team,away_team,home_score,away_score,tournament,neutral,year,home_team_total_points,away_team_total_points,...,away_B2_point,away_B3_point,away_B4_point,away_M1_point,away_M2_point,away_M3_point,away_M4_point,away_T1_point,away_T2_point,away_K1_point
0,36117,1,10,2,1,91,1,2014,603.0,656.0,...,72,71,69,78,77,75,75,80,78,80
1,36121,2,58,1,0,91,1,2014,714.0,529.0,...,59,59,57,72,69,67,63,73,72,66
2,36127,3,1,1,1,91,0,2014,579.0,603.0,...,72,70,70,77,76,72,71,76,74,70
3,36135,4,25,0,3,92,1,2014,500.0,680.0,...,75,74,73,79,78,76,75,87,82,80
4,36137,3,10,1,3,91,0,2014,579.0,656.0,...,72,71,69,78,77,75,75,80,78,80


In [ ]:
#df_new
df_new = df
#len
temp_tor = []
for i in df_old['tournament']:
    temp_tor.append(len(i))
temp_away = []
for i in df_old['away_team']:
    temp_away.append(len(i))
temp_home = []
for i in df_old['home_team']:
    temp_home.append(len(i))
df_new['tor_len'] = temp_tor
df_new['home_len'] = temp_home
df_new['away_len'] = temp_away

# 결측값 있는 행 제거
df_new = df_new.dropna()

df_label1 = df_new['home_score']
df_label2 = df_new['away_score']

In [ ]:
#df_new, df_label 분리 : 모델의 인풋(df_new)과 아웃풋(df_label, home_score 과 away score)
df_new = df_new.drop(['home_score', 'away_score'], axis = 1)
#df_label
df_label = pd.concat([df_label1, df_label2],axis = 1)
df_label.columns = ['home_score', 'away_score']

**df_label** : 왜 인덱스가 7948까지 있을까? 표시는 6547 rows라고 되긴 함

In [ ]:
# concat이 단순하게 합치는 연산이라 index 갱신이 안된듯 

In [ ]:
df_label

,home_score,away_score
0,2,1
1,1,0
2,1,1
3,0,3
4,1,3
...,...,...
2543,0,0
2544,0,5
2545,1,1
2546,0,0


In [ ]:
x_train = df_new.to_numpy()
df_train = pd.DataFrame(data=x_train, columns=df_new.columns)

**df_train**

In [ ]:
df_train

,Unnamed: 0,home_team,away_team,tournament,neutral,year,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,...,away_M1_point,away_M2_point,away_M3_point,away_M4_point,away_T1_point,away_T2_point,away_K1_point,tor_len,home_len,away_len
0,36117,1,10,91,1,2014,603.0,656.0,73,72,...,78,77,75,75,80,78,80,28,4,7
1,36121,2,58,91,1,2014,714.0,529.0,73,72,...,72,69,67,63,73,72,66,28,5,5
2,36127,3,1,91,0,2014,579.0,603.0,72,72,...,77,76,72,71,76,74,70,28,12,4
3,36135,4,25,92,1,2014,500.0,680.0,74,74,...,79,78,76,75,87,82,80,8,6,6
4,36137,3,10,91,0,2014,579.0,656.0,72,72,...,78,77,75,75,80,78,80,28,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,44145,85,16,92,0,2022,1243.8,1335.36,67,66,...,70,69,69,67,72,67,66,8,10,8
2544,44147,82,27,92,0,2022,961.23,1434.68,61,58,...,76,76,75,75,78,75,76,8,7,7
2545,44150,4,34,92,0,2022,1473.04,1405.6,77,77,...,78,74,72,72,75,73,83,8,6,7
2546,44152,55,6,92,0,2022,1425.59,1509.61,77,75,...,80,77,76,76,79,77,78,8,8,5


In [ ]:
#x_train = df_new.to_numpy()
Norm = Normalizer() # 각 변수의 값을 원점으로부터 1만큼 떨어져 있는 값으로 변환 
x_temp = Norm.fit_transform(x_train)
df_temp = pd.DataFrame(data=x_temp, columns=df_new.columns)

**df_temp** : df_train은 x_train을 사용, df_temp는 x_train을 normalize한 값인 x_temp 사용

In [ ]:
df_temp

,Unnamed: 0,home_team,away_team,tournament,neutral,year,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,...,away_M1_point,away_M2_point,away_M3_point,away_M4_point,away_T1_point,away_T2_point,away_K1_point,tor_len,home_len,away_len
0,0.998096,0.000028,0.000276,0.002515,0.000028,0.055657,0.016664,0.018129,0.002017,0.001990,...,0.002156,0.002128,0.002073,0.002073,0.002211,0.002156,0.002211,0.000774,0.000111,0.000193
1,0.998102,0.000055,0.001603,0.002515,0.000028,0.055651,0.019729,0.014617,0.002017,0.001990,...,0.001990,0.001907,0.001851,0.001741,0.002017,0.001990,0.001824,0.000774,0.000138,0.000138
2,0.998135,0.000083,0.000028,0.002514,0.000000,0.055644,0.015997,0.016660,0.001989,0.001989,...,0.002127,0.002100,0.001989,0.001962,0.002100,0.002045,0.001934,0.000774,0.000332,0.000111
3,0.998127,0.000110,0.000691,0.002541,0.000028,0.055631,0.013811,0.018783,0.002044,0.002044,...,0.002182,0.002155,0.002099,0.002072,0.002403,0.002265,0.002210,0.000221,0.000166,0.000166
4,0.998109,0.000083,0.000276,0.002513,0.000000,0.055627,0.015992,0.018119,0.001989,0.001989,...,0.002154,0.002127,0.002072,0.002072,0.002210,0.002154,0.002210,0.000773,0.000331,0.000193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2543,0.998072,0.001922,0.000362,0.002080,0.000000,0.045715,0.028121,0.030191,0.001515,0.001492,...,0.001583,0.001560,0.001560,0.001515,0.001628,0.001515,0.001492,0.000181,0.000226,0.000181
2544,0.998160,0.001854,0.000610,0.002080,0.000000,0.045717,0.021733,0.032438,0.001379,0.001311,...,0.001718,0.001718,0.001696,0.001696,0.001764,0.001696,0.001718,0.000181,0.000158,0.000158
2545,0.997860,0.000090,0.000768,0.002079,0.000000,0.045700,0.033293,0.031769,0.001740,0.001740,...,0.001763,0.001673,0.001627,0.001627,0.001695,0.001650,0.001876,0.000181,0.000136,0.000158
2546,0.997816,0.001243,0.000136,0.002079,0.000000,0.045696,0.032218,0.034117,0.001740,0.001695,...,0.001808,0.001740,0.001718,0.001718,0.001785,0.001740,0.001763,0.000181,0.000181,0.000113


In [ ]:
df_corr = df_temp.corr() # 각 열간의 상관계수 반환 

df_label과 df_temp(normalize된 값)

In [ ]:
y_es = df_label.to_numpy()
x_es = df_temp.to_numpy()
x_t, x_val , y_t, y_val = train_test_split(x_es, y_es, random_state=28)

In [ ]:
params = {
    "estimator__colsample_bylevel": uniform(0.7, 0.3),
    "estimator__gamma": uniform(0, 0.5),
    "estimator__learning_rate": uniform(0.003, 0.3),
    "estimator__max_depth": randint(2, 6),
    "estimator__n_estimators": randint(100, 500), 
    "estimator__subsample": uniform(0.6, 0.4)
}

In [ ]:
#Multioutput regression are regression problems that involve predicting two or more numerical values given an input example.
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror', eval_metric=['mae'], random_state = 1, use_label_encoder=False))

In [ ]:
#search : 하이퍼 파라미터 튜닝
search = RandomizedSearchCV(xgb_model, param_distributions=params, random_state =1, n_iter=5, cv=3,
                            verbose=0, n_jobs=1, return_train_score=True) 
search.fit(x_t, y_t) 

best_param = search.best_params_
print('-')
print(search.best_score_)
print(best_param)
print('-')

-
0.1292525010504187
{'estimator__colsample_bylevel': 0.9147911547905092, 'estimator__gamma': 0.40137875196868245, 'estimator__learning_rate': 0.030840242592221373, 'estimator__max_depth': 3, 'estimator__n_estimators': 101, 'estimator__subsample': 0.9460081007915934}
-


In [ ]:
#RandomizedSearchCV로 찾은 best param을 모델에 적용
xgb_model = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',eval_metric=['mae'], random_state = 1, 
                              use_label_encoder=False, **best_param)).fit(x_t, y_t) 

In [ ]:
#MAE : MSE와 유사한 loss function
#이 코드에서 loss function이 적용되는 부분이 어디지? 못 찾았음
print('MAE: ',mean_absolute_error(y_val, xgb_model.predict(x_val)))

MAE:  0.8653770549050306


In [ ]:
#K-fold 교차 검증:
#전체 훈련 데이터를 K개의 그룹으로 묶고, 그중 한 그룹을 검증셋으로 사용합니다. 
#이때 각각의 모든 데이터 그룹이 적어도 한 번씩은 검증셋으로 사용되어야 합니다. 
#K-fold 교차 검증을 사용함으로써 검정 데이터에 모델이 과적합 되는 것을 막을 수 있습니다. 
folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
scores = cross_val_score(xgb_model, x_val, y_val, scoring='neg_mean_absolute_error', cv=folds, verbose=0)
print('KFold_Neg_MAE: ',np.mean(scores))

KFold_Neg_MAE:  -0.9236260878723336


======================여기까지가 모델 트레이닝========================

# **predict_score 변경 예정**

In [ ]:
#inp_list: 사용자가 입력한 home_team, away team
def predict_score(inp_list, autofill, Norm, seed, model):
    ind_list = []
    for i in inp_list:
        ind_list.append(nat_dic.get(i))
    seed[:,1] = ind_list[0]
    seed[:,2] = ind_list[1]
    x_test = Norm.fit_transform(seed) # Norm.fit_transform() 의 역할은 ? 
    pred = model.predict(x_test)
    return np.ravel(pred).tolist()

# def

In [ ]:
def create_params():
  seed = x_train[random.randint(0,x_train.shape[0])].reshape(1,-1)
  params = {
    'autofill': True,
    'Norm': Norm,
    'seed': seed,
    'model': xgb_model
  }
  return params

In [ ]:
def create_input(home_team, away_team):
  #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
  ind_list = [home_team, away_team]
  return ind_list#input_vector

In [50]:
def select_winning_team(ans, qual): #probability_array
    pred_ans = copy.deepcopy(ans)
    #예선 : 소수점 x, 정수로 승패 판결
    if qual == True :
      #정수
      pred_ans[0] = round(pred_ans[0])
      pred_ans[1] = round(pred_ans[1])
      
    #본선 #소수점 끝까지
    """
    else :
      
      pred_ans[0] = round(pred_ans[0], 1)
      pred_ans[1] = round(pred_ans[1], 1)
    """
    
    if (pred_ans[0]>pred_ans[1]):
          out=0 # home이 우승
    elif (pred_ans[0]<pred_ans[1]):
          out=1 # away가 우승
    elif (pred_ans[0]==pred_ans[1]):
          out=2 #비김

    return out, pred_ans

# 조 편성

In [ ]:

#예선 팀 조 편성 결과 입력
Group_A= ["Netherlands","Senegal","Ecuador","Qatar"]
Group_B= ["England","United States","Iran","Wales"] # United States 가 없음 ;; 
Group_C= ["Argentina","Poland","Mexico", "Saudi Arabia"]
Group_D= ["France","Australia","Tunisia","Denmark"]
Group_E= ["Japan","Spain","Germany","Costa Rica"]
Group_F= ["Morocco","Croatia","Belgium","Canada"]
Group_G= ["Brazil","Switzerland","Cameroon","Serbia"]
Group_H= ["Portugal","South Korea","Uruguay","Ghana"] # South Korea 가 없음 ;; 
Groups={"Group A":Group_A,"Group B":Group_B,"Group C":Group_C,"Group D":Group_D,"Group E":Group_E,"Group F":Group_F,"Group G":Group_G,"Group H":Group_H}


# 32강

In [64]:

##Group stage Matches
print("========Qualifying Games=========")
Group_standings={}
for grp_name in list(Groups.keys()):
    print(f"[{grp_name} Matches]")
    #해당 그룹의 팀들
    probable_countries=Groups[grp_name]

    team_wins_dct={}
    goal_scored_dct={}
    goal_against_dct={}
    win_dct={}
    draw_dct={}
    lost_dct={}
    
    for i in range(len(probable_countries)):
        #team i: team1, team j : team2
        j=i+1
        team_1=probable_countries[i]

        team_wins=0
        
        while j<len((probable_countries)):
            team_2=probable_countries[j]
            team_lst=[team_1,team_2]

            #인풋으로 home_team, away_team만 주면 되는가? 아니면 별도의 벡터 생성 필요?
            input=create_input(team_1, team_2)#np.array([[year,stadium_num,team_1_num,team_2_num]])
            params = create_params()
            #ans : 소수점 ex 1.3 : 1.5
            ans = predict_score(input, **params)

            #qual : 예선이면 true, 본선이면 false
            qual = True
           
            win, prob_lst=select_winning_team(ans, qual)
            #print("applied_score : ", prob_lst,", predicted_score: ", ans)
            
            #본인 팀 스코어
            goal_scored_dct[team_1] = goal_scored_dct.get(team_1,0)+prob_lst[0]
            goal_scored_dct[team_2] = goal_scored_dct.get(team_2,0)+prob_lst[1]
            #상대 팀 스코어
            goal_against_dct[team_1] = goal_against_dct.get(team_1,0)+prob_lst[1]
            goal_against_dct[team_2] = goal_against_dct.get(team_2,0)+prob_lst[0]

            try:
                print(f" {team_1} vs {team_2} \n  {team_lst[win]} wins \n")
                #team1 우승
                if (win)==0:
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+2
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)
                    #team1 1승 추가
                    win_dct[team_1] = win_dct.get(team_1,0)+1
                    win_dct[team_2] = win_dct.get(team_2,0)
                    #team2 1패 추가
                    lost_dct[team_2] = lost_dct.get(team_2,0)+1
                    lost_dct[team_1] = lost_dct.get(team_1,0)
                    #비긴 팀 없음
                    draw_dct[team_2] = draw_dct.get(team_2,0)
                    draw_dct[team_1] = draw_dct.get(team_1,0)

                elif (win)==1:
                    team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+2
                    team_wins_dct[team_1] = team_wins_dct.get(team_1,0)
                    #team2 1승 추가
                    win_dct[team_2] = win_dct.get(team_2,0)+1
                    win_dct[team_1] = win_dct.get(team_1,0)
                    #team1 1패 추가
                    lost_dct[team_1] = lost_dct.get(team_1,0)+1
                    lost_dct[team_2] = lost_dct.get(team_2,0)
                    #비긴 팀 없음
                    draw_dct[team_1] = draw_dct.get(team_1,0)
                    draw_dct[team_2] = draw_dct.get(team_2,0)

            except IndexError:
              #비긴 경우
              print(f"{team_1} vs {team_2} \n  Match Draw \n") 
              team_wins_dct[team_1] = team_wins_dct.get(team_1,0)+1
              team_wins_dct[team_2] = team_wins_dct.get(team_2,0)+1
              
              #비김
              draw_dct[team_1] = draw_dct.get(team_1,0)+1
              draw_dct[team_2] = draw_dct.get(team_2,0)+1
              #승패 없음
              win_dct[team_1] = win_dct.get(team_1,0)
              lost_dct[team_1] = lost_dct.get(team_1,0)
              #승패 없음
              win_dct[team_2] = win_dct.get(team_2,0)
              lost_dct[team_2] = lost_dct.get(team_2,0)


            j=j+1
    
    group_results=[win_dct,draw_dct,lost_dct,team_wins_dct,goal_scored_dct,goal_against_dct]
    Group_standings[grp_name]=group_results


========Qualifying Games=========
[Group A Matches]
 Netherlands vs Senegal 
  Netherlands wins 

 Netherlands vs Ecuador 
  Netherlands wins 

Netherlands vs Qatar 
  Match Draw 

 Senegal vs Ecuador 
  Ecuador wins 

Senegal vs Qatar 
  Match Draw 

 Ecuador vs Qatar 
  Ecuador wins 

[Group B Matches]
 England vs United States 
  England wins 

England vs Iran 
  Match Draw 

England vs Wales 
  Match Draw 

 United States vs Iran 
  United States wins 

United States vs Wales 
  Match Draw 

 Iran vs Wales 
  Wales wins 

[Group C Matches]
Argentina vs Poland 
  Match Draw 

Argentina vs Mexico 
  Match Draw 

Argentina vs Saudi Arabia 
  Match Draw 

Poland vs Mexico 
  Match Draw 

Poland vs Saudi Arabia 
  Match Draw 

 Mexico vs Saudi Arabia 
  Saudi Arabia wins 

[Group D Matches]
 France vs Australia 
  Australia wins 

France vs Tunisia 
  Match Draw 

France vs Denmark 
  Match Draw 

Australia vs Tunisia 
  Match Draw 

Australia vs Denmark 
  Match Draw 

 Tunisia vs Denm

# 16강 진출할 팀 결정

In [45]:
for grp_name in list(Group_standings.keys()):

    team_wins_dct= dict(sorted(Group_standings[grp_name][3].items()))
    goal_scored_dct=dict(sorted(Group_standings[grp_name][4].items()))
    goal_against_dct=dict(sorted(Group_standings[grp_name][5].items()))
    
    win_dct=dict(sorted(Group_standings[grp_name][0].items()))
    draw_dct=dict(sorted(Group_standings[grp_name][1].items()))
    lost_dct=dict(sorted(Group_standings[grp_name][2].items()))
    
    lst_teams=list(team_wins_dct.keys())
    
    win_lst=list(win_dct.values())
    draw_lst=list(draw_dct.values())
    lost_lst=list(lost_dct.values())
    
    lst_win_count=list(team_wins_dct.values())
    goal_scored=list(goal_scored_dct.values())
    goal_against=list(goal_against_dct.values())
    goal_differance=[goal_scored[i]-goal_against[i] for i in range (len(goal_scored))]
    ranking_table=pd.DataFrame(list(zip(lst_teams,win_lst,draw_lst,lost_lst,goal_scored,goal_against,goal_differance,lst_win_count)),columns=["Team","Wins","Draw","Lost","Goal Scored","Goal Against","Goal Differance","Points"])
    ranking_table=ranking_table.sort_values("Points",ascending=False).reset_index(drop=True)
    ranking_table.index = ranking_table.index + 1
    print(f"\n\n{grp_name} Final Rankings")
    print(ranking_table.to_markdown())



Group A Final Rankings
|    | Team        |   Wins |   Draw |   Lost |   Goal Scored |   Goal Against |   Goal Differance |   Points |
|---:|:------------|-------:|-------:|-------:|--------------:|---------------:|------------------:|---------:|
|  1 | Ecuador     |      2 |      1 |      0 |             5 |              3 |                 2 |        5 |
|  2 | Netherlands |      1 |      1 |      1 |             4 |              5 |                -1 |        3 |
|  3 | Qatar       |      1 |      0 |      2 |             4 |              6 |                -2 |        2 |
|  4 | Senegal     |      1 |      0 |      2 |             5 |              4 |                 1 |        2 |


Group B Final Rankings
|    | Team          |   Wins |   Draw |   Lost |   Goal Scored |   Goal Against |   Goal Differance |   Points |
|---:|:--------------|-------:|-------:|-------:|--------------:|---------------:|------------------:|---------:|
|  1 | Wales         |      2 |      1 |      0 | 

# 16강

In [66]:
##Round of 16 Section_1

qualified_teams_1=[]
standings=list(Group_standings.keys())
i=0
print(f"Round of 16\n")
while i < (len(standings)):
    A_team= sorted(Group_standings[standings[i]][3].items(), key=lambda x: x[1], reverse=True)
    team_1=A_team[0][0]
    B_team= sorted(Group_standings[standings[i+1]][3].items(), key=lambda x: x[1], reverse=True)
    team_2=B_team[1][0]
    
    #team_1_num=label_encoder.transform([team_1])[0]
    #team_2_num=label_encoder.transform([team_2])[0]
    team_lst=[team_1,team_2]
    
    #Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
    input=create_input(team_1, team_2)
    params = create_params()
    #res=model.predict(Input_vector)
    ans = predict_score(input, **params)
    #win,_=select_winning_team(res)
    qual = False          
    win, prob_lst=select_winning_team(ans, qual)

    try:
            #print("applied_score : ", prob_lst,", predicted_score: ", ans)
            print(f"{team_1} vs {team_2} \n  {team_lst[win]} wins \n")
            
            #print(f"    {team_lst[win]} into the Quater-Finals  \n")
            qualified_teams_1.append(team_lst[win])
    except IndexError:
            print(f"{team_1} vs {team_2} \n  [ERROR]Match Draw ") 
            """
            winning_team=random.choice(team_lst)
            print(f"   {winning_team} wins at Penaly Shoot-Out ")
            print(f"    {winning_team} into the Quater-Finals  \n")
            qualified_teams_1.append(winning_team)
            """
    i=i+2
    
##Round of 16 Section_2
qualified_teams_2=[]
standings=list(Group_standings.keys())
i=0
while i < (len(standings)):
    A_team= sorted(Group_standings[standings[i]][3].items(), key=lambda x: x[1], reverse=True)
    team_1=A_team[1][0]
    B_team= sorted(Group_standings[standings[i+1]][3].items(), key=lambda x: x[1], reverse=True)
    team_2=B_team[0][0]
    
    #team_1_num=label_encoder.transform([team_1])[0]
    #team_2_num=label_encoder.transform([team_2])[0]
    team_lst=[team_1,team_2]
    
    #Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
    input=create_input(team_1, team_2)
    params = create_params()
    #res=model.predict(Input_vector)
    ans = predict_score(input, **params)
    #win,_=select_winning_team(res)
    qual = False
    win, prob_lst=select_winning_team(ans, qual)

    try:
            #print("applied_score : ", prob_lst,", predicted_score: ", ans)
            print(f"{team_1} vs {team_2} \n  {team_lst[win]} wins \n")
            #print(f"    {team_lst[win]} into the Quater-Finals  \n")
            qualified_teams_2.append(team_lst[win])
            
    except IndexError:
            print(f"{team_1} vs {team_2} \n  [ERROR]Match Draw ") 
            """
            winning_team=random.choice(team_lst)
            print(f"   {winning_team} wins at Penaly Shoot-Out ")
            #print(f"    {winning_team} into the Quater-Finals  \n")
            qualified_teams_2.append(winning_team)
            """
    i=i+2

Round of 16

Netherlands vs Wales 
  Netherlands wins 

Saudi Arabia vs Denmark 
  Denmark wins 

Japan vs Morocco 
  Morocco wins 

Cameroon vs Ghana 
  Ghana wins 

Ecuador vs England 
  Ecuador wins 

Argentina vs Australia 
  Australia wins 

Spain vs Belgium 
  Spain wins 

Brazil vs Portugal 
  Brazil wins 



# 8강

In [67]:
#Quarter Finals

Semifinal_teams=[]
i=0
print(f"Quater Final Matches\n")
while i < (len(qualified_teams_1))-1:
    team_1= qualified_teams_1[i]
    team_2= qualified_teams_1[i+1]
    
    #team_1_num=label_encoder.transform([team_1])[0]
    #team_2_num=label_encoder.transform([team_2])[0]
    team_lst=[team_1,team_2]
    
    #Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
    input=create_input(team_1, team_2)
    params = create_params()
    #res=model.predict(Input_vector)
    ans = predict_score(input, **params)
    #win,_=select_winning_team(res)
    qual = False
    win, prob_lst=select_winning_team(ans, qual)

    try:
            #print("applied_score : ", prob_lst,", predicted_score: ", ans)
            print(f"{team_1} vs {team_2} \n  {team_lst[win]} wins \n")
            #print(f"    {team_lst[win]} into the Semi-Finals \n")
            Semifinal_teams.append(team_lst[win])
            
    except IndexError:
            print(f"{team_1} vs {team_2} \n  [ERROR]Match Draw")
            """
            winning_team=random.choice(team_lst)
            print(f"   {winning_team} wins at Penaly Shoot-Out 🥅🎯")
            print(f"    {winning_team} into the Semi-Finals ⏩⏩ \n")
            Semifinal_teams.append(winning_team)
            """
    i=i+2
    
i=0
while i < (len(qualified_teams_2))-1:
    team_1= qualified_teams_2[i]
    team_2= qualified_teams_2[i+1]
    #team_1_num=label_encoder.transform([team_1])[0]
    #team_2_num=label_encoder.transform([team_2])[0]
    team_lst=[team_1,team_2]
    
    #Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
    input=create_input(team_1, team_2)
    params = create_params()
    #res=model.predict(Input_vector)
    ans = predict_score(input, **params)
    #win,_=select_winning_team(res)
    qual = False
    win, prob_lst=select_winning_team(ans, qual)

    try:
            #print("applied_score : ", prob_lst,", predicted_score: ", ans)
            print(f"{team_1} vs {team_2} \n  {team_lst[win]} wins \n")
            #print(f"    {team_lst[win]} into the Semi-Finals  \n")
            Semifinal_teams.append(team_lst[win])
            
    except IndexError:
            print(f"{team_1} vs {team_2} \n  [ERROR]Match Draw ") 
            """
            winning_team=random.choice(team_lst)
            print(f"   {winning_team} wins at Penaly Shoot-Out 🥅🎯")
            print(f"    {winning_team} into the Semi-Finals ⏩⏩ \n")
            Semifinal_teams.append(winning_team)
            """
    i=i+2

Quater Final Matches

Netherlands vs Denmark 
  Denmark wins 

Morocco vs Ghana 
  Morocco wins 

Ecuador vs Australia 
  Ecuador wins 

Spain vs Brazil 
  Spain wins 



# 4강

In [69]:
final_teams=[]
third_place_match_teams=[]
i=0
print(f"Semi Final Matches\n")
while i < (len(Semifinal_teams))-1:
    team_1= Semifinal_teams[i]
    team_2= Semifinal_teams[i+1]
    
    #team_1_num=label_encoder.transform([team_1])[0]
    #team_2_num=label_encoder.transform([team_2])[0]
    team_lst=[team_1,team_2]
    
    #Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
    input=create_input(team_1, team_2)
    params = create_params()
    #res=model.predict(Input_vector)
    ans = predict_score(input, **params)
    #win,_=select_winning_team(res)
    win, prob_lst=select_winning_team(ans, qual)

    try:
            #print("applied_score : ", prob_lst,", predicted_score: ", ans)
            print(f"{team_1} vs {team_2} \n  {team_lst[win]} wins \n")
            #print(f"    {team_lst[win]} into the FiIFA-Finals ⏩⏩ \n")
            final_teams.append(team_lst[win])
            third_place_match_teams.append(team_lst[(win+1)%2])

            
    except IndexError:
            print(f"{team_1} vs {team_2} \n  [ERROR] Match Draw ") 
            """
            winning_team=random.choice(team_lst)
            print(f"   {winning_team} wins at Penaly Shoot-Out 🥅🎯")
            print(f"    {winning_team} into the FIFA-Finals ⏩⏩ \n")
            final_teams.append(winning_team)
            team_lst.remove(winning_team)
            third_place_match_teams.append(team_lst[0])
            """
    i=i+2

Semi Final Matches

Denmark vs Morocco 
  Denmark wins 

Ecuador vs Spain 
  Ecuador wins 



# 결승

In [72]:
#Finals and Third Place match

print(f"FIFA FINAl\n")
team_1= final_teams[1]
team_2= final_teams[0]
    
#team_1_num=label_encoder.transform([team_1])[0]
#team_2_num=label_encoder.transform([team_2])[0]
team_lst=[team_1,team_2]
    
#Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
input=create_input(team_1, team_2)
params = create_params()
#res=model.predict(Input_vector)
ans = predict_score(input, **params)
#win,_=select_winning_team(res)
win, prob_lst=select_winning_team(ans, qual)

try:
    #print("applied_score : ", prob_lst,", predicted_score: ", ans)
    print("[predicted_score] ", round(ans[0]), ":", round(ans[1]))
    print(f"{team_1} vs {team_2} \n  {team_lst[win]} are the Winners \n\n")
    winner=team_lst[win]
    place_2=team_lst[(win+1)%2]
            
except IndexError:
    print(f"{team_1} vs {team_2} \n  [ERROR]Match Draw ") 
    """
    winning_team=random.choice(team_lst)
    print(f"   {winning_team} wins at Penaly Shoot-Out 🥅🎯")
    print(f"    {winning_team} are the Winners 🎉🏆🎉\n\n")
    winner=winning_team
    
    team_lst.remove(winning_team)
    place_2=team_lst[0]
    """

print(f"Third Place match\n")
team_1= third_place_match_teams[1]
team_2= third_place_match_teams[0]
    
#team_1_num=label_encoder.transform([team_1])[0]
#team_2_num=label_encoder.transform([team_2])[0]
team_lst=[team_1,team_2]
    
#Input_vector=np.array([[year,host_num,team_1_num,team_2_num]])
input=create_input(team_1, team_2)
params = create_params()
#res=model.predict(Input_vector)
ans = predict_score(input, **params)
#win,_=select_winning_team(res)
win, prob_lst=select_winning_team(ans, qual)

try:
    print(f"{team_1} vs {team_2} \n  {team_lst[win]} Wins the 3rd Place \n")
    place_3=team_lst[win]
            
except IndexError:
    print(f"{team_1} vs {team_2} \n  [ERROR]Match Draw ") 
    """
    winning_team=random.choice(team_lst)
    print(f"   {winning_team} wins at Penaly Shoot-Out 🥅🎯")
    print(f"    {winning_team} Wins the 3rd Place 🎉🏆🎉\n")
    place_3=winning_team
    """
    

    
print(f"\n\nWinner is {winner} ")
print(f"Runner-up is {place_2} ")
print(f"3rd Place is {place_3} ")

FIFA FINAl

[predicted_score]  2 : 1
Ecuador vs Denmark 
  Ecuador are the Winners 


Third Place match

Spain vs Morocco 
  Morocco Wins the 3rd Place 



Winner is Ecuador 
Runner-up is Denmark 
3rd Place is Morocco 
